In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from tqdm import tqdm
from khaiii import KhaiiiApi

In [2]:
data_dic = {}
idx = 0

for curDir, dirs, files in tqdm(os.walk("./NewsData")):
    for f in files:
        try:
            path = os.path.join(curDir, f)
            data = pd.read_csv(path, encoding='cp949')
            data_dic[idx] = data
        except:
            path = os.path.join(curDir, f)
            print(f'========={path} ERROR=========')
        idx += 1

381it [01:05,  5.85it/s]


In [3]:
df = pd.concat(data_dic).reset_index(drop=True)
df['Date'] = df['Date'].apply(lambda x: x.split('.')[0] + '-' + x.split('.')[1] + '-' + x.split('.')[2])
df['Date'] = pd.to_datetime(df['Date'])

df['Title_Article'] = df['Title'] + df['Article']
df = df.drop(['Title', 'Article'], axis=1)

In [4]:
# 명사, 동사, 형용사

api = KhaiiiApi()
word_lst = []

for text in tqdm(df['Title_Article']):
    morph_lst = []    
    
    for word in api.analyze(text):
        for morph in word.morphs:
            if morph.tag in ['NNG', 'NNP', 'VV', 'VA', 'MAG'] and len(morph.lex) != 1:
                morph_lst.append(morph.lex)
        
    word_lst.append(morph_lst)

100%|██████████| 1584147/1584147 [8:18:37<00:00, 52.95it/s]   


In [ ]:
# 명사

api = KhaiiiApi()
word_lst = []

for text in tqdm(df['Title_Article']):
    morph_lst = []    
    
    try:
        for word in api.analyze(text):
            for morph in word.morphs:
                if morph.tag in ['NNG', 'NNP'] and len(morph.lex) != 1:
                    morph_lst.append(morph.lex)

        word_lst.append(morph_lst)
    except:
        word_lst.append(morph_lst)

In [6]:
clean_data = []

for text in tqdm(word_lst):
    clean_text = ' '.join(text)
    clean_data.append(clean_text)

100%|██████████| 1584147/1584147 [00:09<00:00, 163245.72it/s]


In [7]:
df['Title_Article'] = clean_data

In [15]:
df.to_csv('Khaiii.csv', index=False)

In [14]:
df

,Date,Category,Media,Title_Article
0,2021.02.19. 오후 11:27,증권,한국경제TV,증권 하나은행 옵티머스 제재심 결론 못내 내달 속개 이민재 금융 감독원 옵티머스 자...
1,2021.02.19. 오후 11:17,증권,한국경제,대어 유망 스팩 총정리 주코 노미 미국 증시 기업 수목적회사 열풍 더욱 뜨거워지 특...
2,2021.02.19. 오후 10:04,증권,아시아경제,장단기 금리 최대 개인 순매 수세 지속 이민지 장단기 금리 최대 수준 벌어지 경기 ...
3,2021.02.19. 오후 9:59,증권,매일경제,블랙록 카카오 지분 매수전 세계 최대 규모 자산 운용사 블랙록자산 운용 한국 플랫폼...
4,2021.02.19. 오후 9:51,증권,아이뉴스24,금감원 옵티머스 제재심 결론 내달 재심 투자 증권 하나은행 대상 금융 감독원 옵티머...
...,...,...,...,...
1584142,2020.09.24. 오전 12:00,증권,로이터 증권(신),스타벅스 수정 작성 회사 달러 신용 계약 현재 로이터 스타 벅스 주식회사 스타벅 주...
1584143,2020.09.24. 오전 12:00,증권,로이터 증권(신),분기 손실 로이터 주식회사 에어 분기 결과 보고
1584144,2020.09.24. 오전 12:00,증권,로이터 증권(신),옴니브리지 홀딩스 유닛 중국온홀딩스 그룹 함께 진입선전 로이터 옴니브 리지 홀딩스 ...
1584145,2020.09.24. 오전 12:00,증권,로이터 증권(신),화요일 아침 회사 채무자 파산 법원 의거 조직 개편 공동 계획안 제출 로이터 화요일...
